# Defining `ufuncs` using `vectorize`

You have been able to define your own NumPy [`ufuncs`](http://docs.scipy.org/doc/numpy/reference/ufuncs.html) for quite some time, but it's a little involved.  

You can read through the [documentation](http://docs.scipy.org/doc/numpy/user/c-info.ufunc-tutorial.html), the example they post there is a ufunc to perform 

$$f(a) = \log \left(\frac{a}{1-a}\right)$$

It looks like this:

```c
static void double_logit(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char *in = args[0], *out = args[1];
    npy_intp in_step = steps[0], out_step = steps[1];

    double tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(double *)in;
        tmp /= 1-tmp;
        *((double *)out) = log(tmp);
        /*END main ufunc computation*/

        in += in_step;
        out += out_step;
    }
}
```

And **note**, that's just for a `double`.  If you want `floats`, `long doubles`, etc... you have to write all of those, too.  And then create a `setup.py` file to install it.  And I left out a bunch of boilerplate stuff to set up the import hooks, etc...

# Say "thank you" to the NumPy devs

We can use Numba to define ufuncs without all of the pain.

In [37]:
import numpy
import math

Let's define a function that operates on two inputs

In [38]:
def trig(a, b):
    return math.sin(a**2) * math.exp(b)

In [39]:
trig(1, 1)

2.2873552871788423

Seems reasonable.  However, the `math` library only works on scalars.  If we try to pass in arrays, we'll get an error.

In [40]:
a = numpy.ones((5,5))
b = numpy.ones((5,5))

In [41]:
trig(a, b)

TypeError: only length-1 arrays can be converted to Python scalars

In [42]:
from numba import vectorize

In [43]:
vec_trig = vectorize()(trig)

In [44]:
vec_trig(a, b)

array([[ 2.28735529,  2.28735529,  2.28735529,  2.28735529,  2.28735529],
       [ 2.28735529,  2.28735529,  2.28735529,  2.28735529,  2.28735529],
       [ 2.28735529,  2.28735529,  2.28735529,  2.28735529,  2.28735529],
       [ 2.28735529,  2.28735529,  2.28735529,  2.28735529,  2.28735529],
       [ 2.28735529,  2.28735529,  2.28735529,  2.28735529,  2.28735529]])

And just like that, the scalar function `trig` is now a NumPy `ufunc` called `vec_trig`

Note that this is a "Dynamic UFunc" with no signature given.  

How does it compare to just using NumPy?  Let's check

In [45]:
def numpy_trig(a, b):
    return numpy.sin(a**2) * numpy.exp(b)

In [46]:
a = numpy.random.random((1000, 1000))
b = numpy.random.random((1000, 1000))

In [47]:
%timeit vec_trig(a, b)

10 loops, best of 3: 34.4 ms per loop


In [48]:
%timeit numpy_trig(a, b)

10 loops, best of 3: 34.9 ms per loop


What happens if we do specify a signature?  Is there a speed boost?

In [49]:
vec_trig = vectorize('float64(float64, float64)')(trig)

In [50]:
%timeit vec_trig(a, b)

10 loops, best of 3: 34.5 ms per loop


No, not really.  But(!), if we have a signature, then we can add the target `kwarg`.

In [51]:
vec_trig = vectorize('float64(float64, float64)', target='parallel')(trig)

In [52]:
%timeit vec_trig(a, b)

100 loops, best of 3: 11.8 ms per loop


Automatic multicore operations!

**Note**: `target='parallel'` is not always the best option.  There is overhead in setting up the threading, so if the individual scalar operations that make up a `ufunc` are simple you'll probably get better performance in serial.  If the individual operations are more expensive (like trig!) then parallel is (usually) a good option.

### Passing multiple signatures

If you use multiple signatures, they have to be listed in order of most specific -> least specific

In [53]:
@vectorize(['int32(int32, int32)',
            'int64(int64, int64)',
            'float32(float32, float32)',
            'float64(float64, float64)'])
def trig(a, b):
    return math.sin(a**2) * math.exp(b)

In [54]:
trig(1, 1)

2

In [55]:
trig(1., 1.)

2.2873552871788423

In [56]:
trig.ntypes

4

## [Exercise: Clipping an array](./exercises/07.Vectorize.Exercises.ipynb#Exercise:-Clipping-an-array)

Yes, NumPy has a `clip` ufunc already, but let's pretend it doesn't.  

Create a Numba vectorized ufunc that takes a vector `a`, a lower limit `amin` and an upper limit `amax`.  It should return the vector `a` with all values clipped such that $a_{min} < a < a_{max}$:

In [68]:
# %load snippets/clip.py
def truncate(a, amin, amax):
    if a < amin:
        a = amin
    elif a > amax:
        a = amax
    return a

vec_truncate_serial = vectorize(['float64(float64, float64, float64)'])(truncate)
vec_truncate_par = vectorize(['float64(float64, float64, float64)'], target='parallel')(truncate)


In [69]:
a = numpy.random.random((5000))

In [70]:
amin = .2
amax = .6

In [71]:
%timeit vec_truncate_serial(a, amin, amax)

The slowest run took 9.46 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.12 µs per loop


In [72]:
%timeit vec_truncate_par(a, amin, amax)

The slowest run took 11.24 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.8 µs per loop


In [73]:
%timeit numpy.clip(a, amin, amax)

The slowest run took 4.54 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.17 µs per loop


In [74]:
a = numpy.random.random((100000))

In [75]:
%timeit vec_truncate_serial(a, amin, amax)

1000 loops, best of 3: 489 µs per loop


In [76]:
%timeit vec_truncate_par(a, amin, amax)

1000 loops, best of 3: 295 µs per loop


In [77]:
%timeit numpy.clip(a, amin, amax)

1000 loops, best of 3: 203 µs per loop


## [Exercise: Create `logit` ufunc](./exercises/07.Vectorize.Exercises.ipynb#Exercise:-Create-logit-ufunc)

Recall from above that this is a ufunc which performs this operation:

$$f(a) = \log \left(\frac{a}{1-a}\right)$$

In [90]:
# %load snippets/logit.py
@vectorize(['float64(float64)'])
def logit(a):
    return math.log(a / (1 - a))

In [91]:
a = numpy.linspace(.1, .9, 9)

In [92]:
logit(a)

array([-2.19722458, -1.38629436, -0.84729786, -0.40546511,  0.        ,
        0.40546511,  0.84729786,  1.38629436,  2.19722458])

## Performance of `vectorize` vs. regular array-wide operations

In [107]:
@vectorize
def discriminant(a, b, c):
    return b**2 - 4 * a * c

In [108]:
a = numpy.arange(10000)
b = numpy.arange(10000)
c = numpy.arange(10000)

In [109]:
%timeit discriminant(a, b, c)

The slowest run took 6298.38 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 11 µs per loop


In [106]:
%timeit b**2 - 4 * a * c

The slowest run took 11.97 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 38.5 µs per loop


What's going on?

* Each array operation creates a temporary copy
* Each of these arrays are loaded into and out of cache a whole bunch

Diferencia:
* Sin @vectorize, la operación tarda 38e-6 s aprox.
* Con @vectorize, la función demora 11e-6 s aprox.

In [97]:
del a, b, c